In [1]:
import csv
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
import pickle

print(tf.__version__)

2.2.0


In [2]:
df = pd.read_csv ('Dataset1.csv')
print (df)

                                        INTENT  ID Unnamed: 2
0                       Undo the last sentence   1        NaN
1                           Undo the last word   1        NaN
2               Can you undo the last sentence   1        NaN
3                         Please undo the text   1        NaN
4                       Undo the selected text   1        NaN
..                                         ...  ..        ...
423             Please delete all text in bold  26        NaN
424             Kindly delete all text in bold  26        NaN
425                   Erase the last paragraph  26        NaN
426  Erase the last two words of that sentence  26        NaN
427                      Please erase all text  26        NaN

[428 rows x 3 columns]


In [3]:
df_shuffled = df.sample(frac=1).reset_index(drop=True)

In [11]:
for index, row in df_shuffled.iterrows():

    tts_string=row['INTENT']
    tts_string=tts_string.lower() #lower case

    #lemmatize, then remove stopwords
    lemmatizer=WordNetLemmatizer()
    lemmatised_string=word_tokenize(tts_string)
    tts_string=""
    for word in lemmatised_string:
        tts_string=tts_string+lemmatizer.lemmatize(word)+" "
    tts_string=tts_string.strip()
    print(tts_string)

    #remove stopwords
    tokens = word_tokenize(tts_string)
    result = [i for i in tokens if not i in STOPWORDS]
    
    
    custom_stopwords = ['please','could','kindly']
    # using list comprehension to perform task 
    result = [i for i in result if i not in custom_stopwords] 


    print(result)
    
    preprocessed_str=""
#     print("here",preprocessed_str)
    for i in result:
        preprocessed_str=preprocessed_str+" "+i
#     preprocessed_str=' '.join(map(str, result))  
    print("Pre processed string",preprocessed_str)
    preprocessed_str=preprocessed_str.strip()
    # row['INTENT']=preprocessed_string

    #remove certain words
    #lemmatization
    #convert digits to numerals
    #text canonicalization?

    df_shuffled.at[index,'INTENT']=preprocessed_str

move line left
['move', 'line', 'left']
Pre processed string  move line left
display command
['display', 'command']
Pre processed string  display command
centre align icon
['centre', 'align', 'icon']
Pre processed string  centre align icon
undo previous sentence
['undo', 'previous', 'sentence']
Pre processed string  undo previous sentence
left align image
['left', 'align', 'image']
Pre processed string  left align image
remove underline word red color
['remove', 'underline', 'word', 'red', 'color']
Pre processed string  remove underline word red color
display help ?
['display', 'help', '?']
Pre processed string  display help ?
remove formatting sentence
['remove', 'formatting', 'sentence']
Pre processed string  remove formatting sentence
position table middle
['position', 'table', 'middle']
Pre processed string  position table middle
bold last two word
['bold', 'last', 'two', 'word']
Pre processed string  bold last two word
change subscript formatingstyle
['change', 'subscript', 'forma

undo selected text
['undo', 'selected', 'text']
Pre processed string  undo selected text
format italic
['format', 'italic']
Pre processed string  format italic
pause dictation near bold text
['pause', 'dictation', 'near', 'bold', 'text']
Pre processed string  pause dictation near bold text
make selected text italic
['make', 'selected', 'text', 'italic']
Pre processed string  make selected text italic
remove last two character subscript
['remove', 'last', 'two', 'character', 'subscript']
Pre processed string  remove last two character subscript
format selected text italic
['format', 'selected', 'text', 'italic']
Pre processed string  format selected text italic
remove formatting 5 line
['remove', 'formatting', '5', 'line']
Pre processed string  remove formatting 5 line
unitalicize selected text
['unitalicize', 'selected', 'text']
Pre processed string  unitalicize selected text
remove formatting
['remove', 'formatting']
Pre processed string  remove formatting
left align image
['left', 'a

In [12]:
vocab_size = 1000
embedding_dim = 20
max_length = 10
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_portion = 0.8

In [13]:
train_size = int(428*training_portion)
X=np.array(df_shuffled['INTENT'].to_list())
Y_arr=np.array(df_shuffled['ID'].to_list())
Y=[]
for val in Y_arr:
    d=[val]
    Y.append(d)
    
train_X = X[0: train_size]
train_Y = Y[0: train_size]

validation_X = X[train_size:]
validation_Y = Y[train_size:]

print(train_size)
print(len(train_X))
print(len(train_Y))
print(train_Y)
print(len(validation_X))
print(len(validation_Y))

342
342
342
[[16], [24], [14], [1], [16], [7], [25], [18], [14], [2], [10], [1], [1], [5], [1], [3], [17], [1], [12], [15], [8], [3], [6], [13], [26], [26], [22], [19], [14], [6], [6], [6], [18], [19], [6], [15], [17], [26], [21], [26], [12], [7], [23], [17], [17], [24], [11], [21], [1], [17], [23], [24], [16], [24], [14], [5], [13], [3], [2], [9], [7], [26], [18], [18], [14], [6], [18], [14], [8], [6], [8], [18], [12], [5], [17], [24], [25], [13], [19], [2], [15], [5], [21], [4], [23], [3], [3], [13], [4], [6], [25], [12], [12], [14], [13], [8], [13], [8], [6], [3], [20], [2], [2], [24], [11], [24], [13], [18], [4], [24], [19], [15], [5], [1], [9], [6], [14], [12], [20], [19], [17], [10], [4], [1], [20], [11], [19], [5], [26], [11], [4], [23], [18], [9], [24], [2], [1], [12], [12], [23], [24], [24], [18], [18], [3], [4], [15], [11], [6], [17], [25], [25], [5], [22], [6], [15], [15], [23], [26], [24], [23], [14], [2], [10], [22], [5], [9], [7], [4], [6], [3], [3], [22], [22], [13], [17

In [14]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_X)
word_index = tokenizer.word_index
dict(list(word_index.items())[0:10])

{'<OOV>': 1,
 'last': 2,
 'remove': 3,
 'word': 4,
 'text': 5,
 'selected': 6,
 'align': 7,
 'bold': 8,
 'two': 9,
 'underline': 10}

In [15]:
train_sequences = tokenizer.texts_to_sequences(train_X)
print(train_sequences[1])

[39, 22]


In [16]:

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(len(train_sequences[0]))
print(len(train_padded[0]))

print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[10]))
print(len(train_padded[10]))

3
10
2
10
3
10


In [17]:
validation_sequences = tokenizer.texts_to_sequences(validation_X)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(len(validation_sequences))
print(validation_padded.shape)

86
(86, 10)


In [18]:
validation_Y=np.array(validation_Y)
training_Y=np.array(train_Y)
print(validation_Y.shape)
print(training_Y.shape)

(86, 1)
(342, 1)


In [30]:
model = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=10),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
#    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
#     use ReLU in place of tanh function since they are very good alternatives of each other.
#     tf.keras.layers.Dense(embedding_dim, activation='relu'),
    # Add a Dense layer with 27(26+0) units and softmax activation.
    # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
    tf.keras.layers.Dense(27, activation='softmax')
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 20)            20000     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 40)                6560      
_________________________________________________________________
dense_2 (Dense)              (None, 27)                1107      
Total params: 27,667
Trainable params: 27,667
Non-trainable params: 0
_________________________________________________________________


print(modaael.inputLayers)

In [20]:
print(model.inputLayers)

AttributeError: 'Sequential' object has no attribute 'inputLayers'

In [32]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 150
history = model.fit(train_padded, training_Y, epochs=num_epochs, validation_data=(validation_padded, validation_Y), verbose=2)

Epoch 1/150
11/11 - 1s - loss: 0.0499 - accuracy: 1.0000 - val_loss: 0.2625 - val_accuracy: 0.9186
Epoch 2/150
11/11 - 0s - loss: 0.0429 - accuracy: 1.0000 - val_loss: 0.2106 - val_accuracy: 0.9419
Epoch 3/150
11/11 - 0s - loss: 0.0384 - accuracy: 1.0000 - val_loss: 0.2182 - val_accuracy: 0.9419
Epoch 4/150
11/11 - 0s - loss: 0.0349 - accuracy: 1.0000 - val_loss: 0.2170 - val_accuracy: 0.9419
Epoch 5/150
11/11 - 0s - loss: 0.0312 - accuracy: 1.0000 - val_loss: 0.2185 - val_accuracy: 0.9535
Epoch 6/150
11/11 - 0s - loss: 0.0288 - accuracy: 1.0000 - val_loss: 0.2282 - val_accuracy: 0.9535
Epoch 7/150
11/11 - 0s - loss: 0.0260 - accuracy: 1.0000 - val_loss: 0.2187 - val_accuracy: 0.9535
Epoch 8/150
11/11 - 0s - loss: 0.0238 - accuracy: 1.0000 - val_loss: 0.2137 - val_accuracy: 0.9535
Epoch 9/150
11/11 - 0s - loss: 0.0220 - accuracy: 1.0000 - val_loss: 0.2132 - val_accuracy: 0.9419
Epoch 10/150
11/11 - 0s - loss: 0.0203 - accuracy: 1.0000 - val_loss: 0.2104 - val_accuracy: 0.9535
Epoch 11/

11/11 - 0s - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.2551 - val_accuracy: 0.9419
Epoch 84/150
11/11 - 0s - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.2560 - val_accuracy: 0.9302
Epoch 85/150
11/11 - 0s - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.2571 - val_accuracy: 0.9419
Epoch 86/150
11/11 - 0s - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.2603 - val_accuracy: 0.9302
Epoch 87/150
11/11 - 0s - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.2566 - val_accuracy: 0.9535
Epoch 88/150
11/11 - 0s - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.2622 - val_accuracy: 0.9302
Epoch 89/150
11/11 - 0s - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.2542 - val_accuracy: 0.9535
Epoch 90/150
11/11 - 0s - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.2666 - val_accuracy: 0.9302
Epoch 91/150
11/11 - 0s - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.2572 - val_accuracy: 0.9302
Epoch 92/150
11/11 - 0s - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.2455 - val_accuracy: 0.9302
Epoch 93/150


In [22]:
accr = model.evaluate(validation_padded,validation_Y)
# print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

3/3 [==============================] - 0s 3ms/step - loss: 0.4313 - accuracy: 0.8953


In [23]:
# new_commands = [['please stop my dictation here'],['subscript the last digit'],['superscript'],['remove italics here'],['end my bullet list'],['can you display all commands'],['help me'],['pause my dictation'],['please remove that bold'],['undo that'],['can you delete the last word'],['remove that strikethrough'],['centre align the content please'],['can you right align this image'],['could you remove formatting for the last word'],['insert a comment here'],['remove underline for last sentence'],['superscript that'],['can you remove subscript from here'],['Please go to next bullet'],['remove subscript'],['Start inserting bullet'],['please open commands'],['can you undo this'],['remove the underline here'],['align content left'],['please bold that'],['underline please'],['right align the text please'],['can you make it in italics'],['please make this italic'],['can you remove the subscript'],['move text right'],['strikethrough the last text'],['show help please'],['insert a bullet here']]
new_commands = [['hey underline the last word'],['remove italics'],['Stop my dictation'],['insert a bullet here'],['MOve content to right']]


In [24]:
new_commands_processed=[]
    
for i,ele in enumerate(new_commands):

    tts=ele[0]
    print(tts)
    tts=tts.lower() #lower case

    #lemmatize, then remove stopwords
    lemmatizer=WordNetLemmatizer()
    lemmatised_string=word_tokenize(tts)
    tts=""
    for word in lemmatised_string:
        tts=tts+lemmatizer.lemmatize(word)+" "
    tts=tts.strip()
    # print(tts_string)

    #remove stopwords
    tokens = word_tokenize(tts)
    result = [i for i in tokens if not i in STOPWORDS]
    
    
    custom_stopwords = ['please','could','kindly']
    # using list comprehension to perform task 
    result = [i for i in result if i not in custom_stopwords] 

    
    print(result)
    preprocessed_string=""
    for i in result:
        preprocessed_string=preprocessed_string+" "+i
#     preprocessed_string= ' '.join(map(str, result)) 
    print(preprocessed_string)
    preprocessed_string=preprocessed_string.strip()
    str0=[preprocessed_string]
#     d=[val]
    new_commands_processed.append(str0)
#     new_commands[i]=preprocessed_string
    # row['INTENT']=preprocessed_string

    #remove certain words
    #lemmatization
    #convert digits to numerals
    #text canonicalization?

    

# print(new_commands')
    

hey underline the last word
['hey', 'underline', 'last', 'word']
 hey underline last word
remove italics
['remove', 'italic']
 remove italic
Stop my dictation
['stop', 'dictation']
 stop dictation
insert a bullet here
['insert', 'bullet']
 insert bullet
MOve content to right
['move', 'content', 'right']
 move content right


In [25]:
label={'0':'','1':'UNDO','2':'BOLD','3':'REMOVE_BOLD','4':'ITALIC','5':'REMOVE_ITALIC','6':'UNDERLINE','7':'REMOVE_UNDERLINE','8':'SUPERSCRIPT','9':'REMOVE_BOLD','10':'SUBSCRIPT','11':'REMOVE_SUBSCRIPT','12':'STRIKETHROUGH','13':'REMOVE_STRIKETHROUGH','14':'ALIGN_CENTER','15':'INSERT_COMMENT','16':'ALIGN_LEFT','17':'ALIGN_RIGHT','18':'REMOVE_FORMATTING','19':'INSERT_BULLET','20':'NEXT_BULLET','21':'END_BULLET','22':'PAUSE_DICTATION','23':'STOP_DICTATION','24':'SHOW_ALL_COMMANDS','25':'SHOW_HELP','26':'DELETE'}
label_list=list_values = [ v for v in label.values() ]
# print(new_commands_processed)
for dum in new_commands_processed :
    print(dum)
    sequences = tokenizer.texts_to_sequences(dum)
#     print(sequences)
    padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    print(padded)
    # seq = tokenizer.texts_to_sequences()
    # padded = pad_sequences(seq, maxlen=max_length)
    # print(padded)
    pred = model.predict(padded)
    print(label_list[np.argmax(pred)],"\n")
#     print(np.argmax(pred),"\n")
#     print(pred)

['hey underline last word']
[[ 1 10  2  4  0  0  0  0  0  0]]
UNDERLINE 

['remove italic']
[[ 3 29  0  0  0  0  0  0  0  0]]
REMOVE_ITALIC 

['stop dictation']
[[23 12  0  0  0  0  0  0  0  0]]
STOP_DICTATION 

['insert bullet']
[[27 13  0  0  0  0  0  0  0  0]]
INSERT_BULLET 

['move content right']
[[44 57 24  0  0  0  0  0  0  0]]
ALIGN_RIGHT 



In [22]:
# model.save('weights/lstm_4')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: weights/lstm_4/assets


In [26]:

import tensorflow.keras as keras

tf.keras.models.save_model(model,"weights/sigmoid")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: weights/sigmoid/assets


In [25]:

# Converting a SavedModel to a TensorFlow Lite model.
converter = tf.lite.TFLiteConverter.from_saved_model("weights/lstm_4")
tflite_model = converter.convert()


In [28]:
# Save the TF Lite model.
with tf.io.gfile.GFile('tflite/lstm_4.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Converting into CoreML model

In [29]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="tflite/lstm_4.tflite")
interpreter.allocate_tensors()

# Get tensor details.
details = interpreter.get_tensor_details()

for detail in details:
    print(detail['index'], detail['name'], detail['shape'])

0 embedding_1_input [ 1 10]
1 Func/StatefulPartitionedCall/input/_2 [1000   20]
2 sequential_1/bidirectional_1/forward_lstm_1/zeros;StatefulPartitionedCall/sequential_1/bidirectional_1/forward_lstm_1/zeros [ 1 20]
3 Func/StatefulPartitionedCall/sequential_1/bidirectional_1/backward_lstm_1/StatefulPartitionedCall/input/_32 [80]
4 Func/StatefulPartitionedCall/sequential_1/bidirectional_1/forward_lstm_1/StatefulPartitionedCall/input/_19 [80]
5 TensorArrayV2_1;StatefulPartitionedCall/sequential_1/bidirectional_1/forward_lstm_1/StatefulPartitionedCall/TensorArrayV2_1 [10  1 20]
6 strided_slice;StatefulPartitionedCall/sequential_1/bidirectional_1/forward_lstm_1/StatefulPartitionedCall/strided_slice []
7 StatefulPartitionedCall/sequential_1/dense_2/BiasAdd/ReadVariableOp [27]
8 TensorArrayV2Write/TensorListSetItem []
9 TensorArrayV2Write/TensorListSetItem1 [1]
10 TensorArrayV2Write/TensorListSetItem2 []
11 TensorArrayV2Write/TensorListSetItem3 [1]
12 TensorArrayV2Write/TensorListSetItem4 []
1

In [30]:
def get_variable(interpreter, index, transposed=False):
    var = interpreter.get_tensor(index)
    if transposed:
        var = np.transpose(var, (1, 0))    
    return var

In [32]:
#We are going to directly convert our .pb file and using tf2


# imports ...
FROZEN_MODEL_FILE = 'weights/lstm_4/saved_model.pb'

# Taken from https://stackoverflow.com/a/52823701/4708657
def freeze_graph(graph, session, output):
    with graph.as_default():
        graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen = tf.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, ".", FROZEN_MODEL_FILE, as_text=False)


# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="tflite/lstm_4.tflite")
interpreter.allocate_tensors()

input_shape = [28, 28, 1]
model = network(input_shape, interpreter)

# FREEZE GRAPH
session = tf.keras.backend.get_session()

INPUT_NODE = model.inputs[0].op.name
OUTPUT_NODE = model.outputs[0].op.name
freeze_graph(session.graph, session, [out.op.name for out in model.outputs])

NameError: name 'network' is not defined

In [46]:
import tfcoreml
import tensorflow
print(tf.__version__)

2.2.0


In [52]:
model.save('weights/abc') 

INFO:tensorflow:Assets written to: weights/abc/assets


INFO:tensorflow:Assets written to: weights/abc/assets


In [27]:
new_model = tf.keras.models.load_model('weights/sigmoid')

# Check its architecture
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 20)            20000     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 40)                6560      
_________________________________________________________________
dense_1 (Dense)              (None, 27)                1107      
Total params: 27,667
Trainable params: 27,667
Non-trainable params: 0
_________________________________________________________________


In [28]:
# mod0=tf.keras.models.load_model(model,"weights/lstm_4")
model = tfcoreml.convert(tf_model_path='./weights/lstm_4/saved_model.pb',
                         mlmodel_path='./model.mlmodel',
                         input_name_shape_dict={'input_1': (1, 10)},
                         output_feature_names=['Identity'])

NameError: name 'tfcoreml' is not defined

In [29]:
from tensorflow.keras.applications import ResNet50
import tfcoreml

# keras_model = ResNet50(weights=None, input_shape=(224, 224, 3))
# keras_model.save('./weights/lstm_4/saved_model.pb')

# print input shape
print(model.input_shape)

# get input, output node names for the TF graph from the Keras model
input_name = new_model.inputs[0].name.split(':')[0]
keras_output_node_name = new.outputs[0].name.split(':')[0]
graph_output_node_name = keras_output_node_name.split('/')[-1]

model = tfcoreml.convert('./weights/sigmoid',
                         input_name_shape_dict={input_name: (1, 224, 224, 3)},
                         output_feature_names=[graph_output_node_name])


model.save('./model.mlmodel')

(None, 10)


NameError: name 'new' is not defined

In [65]:
print(tf.__version__)

2.2.0
